In [13]:
# from pathlib import Path
# import pandas as pd
# import json
# import pickle
# from mirar.processors.sources.source_loader import load_source_table
# from mirar.processors.sources.source_exporter import save_source_table
# from mirar.data.utils import decode_img
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# from astropy import units as u
# from astropy import visualization
# from astropy.io import fits
# from astropy.time import Time
from matplotlib.colors import Normalize
from matplotlib.ticker import MultipleLocator
from matplotlib.backends.backend_pdf import PdfPages
import fastavro
from fastavro import reader
# %matplotlib inline

from winterdrb.paths import get_combined_avro_path, get_pdf_path
from winterdrb.utils import parse_night_data
from winterdrb.plot import generate_single_page
from winterdrb.models import RealBogus
from winterdrb.filtering import apply_clean_filter
from winterdrb.database.populate import populate_night
from tqdm import tqdm

# from winterdrb.filtering

In [14]:
night = "20230913"
res = parse_night_data(night)

TypeError: populate_night() missing 1 required positional argument: 'ann_fields'

In [4]:
print([x for x in res.columns])

['objectid', 'cutout_science', 'cutout_template', 'cutout_difference', 'prv_candidates', 'candid', 'deprecated', 'jd', 'fid', 'exptime', 'ndethist', 'jdstarthist', 'jdendhist', 'progname', 'programid', 'isdiffpos', 'field', 'ra', 'dec', 'magzpsci', 'magzpsciunc', 'magzpscirms', 'diffmaglim', 'magpsf', 'sigmapsf', 'chipsf', 'magap', 'sigmagap', 'magapbig', 'sigmagapbig', 'magdiff', 'magfromlim', 'distnr', 'magnr', 'sigmagnr', 'xpos', 'ypos', 'sky', 'fwhm', 'classtar', 'mindtoedge', 'seeratio', 'aimage', 'bimage', 'aimagerat', 'bimagerat', 'elong', 'nneg', 'nbad', 'sumrat', 'dsnrms', 'ssnrms', 'dsdiff', 'scorr', 'clrcoeff', 'clrcounc', 'zpclrcov', 'zpmed', 'clrmed', 'clrrms', 'rb', 'rbversion', 'ssdistnr', 'ssmagnr', 'ssnamenr', 'tooflag', 'nmtchps', 'psra1', 'psdec1', 'psobjectid1', 'sgmag1', 'srmag1', 'simag1', 'szmag1', 'sgscore1', 'distpsnr1', 'psobjectid2', 'sgmag2', 'srmag2', 'simag2', 'szmag2', 'sgscore2', 'distpsnr2', 'psobjectid3', 'sgmag3', 'srmag3', 'simag3', 'szmag3', 'sgscor

In [5]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(res.iloc[0])

objectid                                                   WNTR23aaasb
cutout_science       b'\x1f\x8b\x08\x00\xc7\xc7\x08e\x02\xff\xec\xb...
cutout_template      b'\x1f\x8b\x08\x00\xc7\xc7\x08e\x02\xff\xec\xb...
cutout_difference    b'\x1f\x8b\x08\x00\xc7\xc7\x08e\x02\xff\xec\xb...
prv_candidates                    jd     magpsf  sigmapsf
0  2.4602...
candid                                                          100304
deprecated                                                       False
jd                                                      2460201.737576
fid                                                                  2
exptime                                                          600.0
ndethist                                                             0
jdstarthist                                             2460201.737576
jdendhist                                               2460201.737576
progname                                                      2023A007
progra

In [6]:
# mag_diff = 0.5

# # masks = [
# #     res["nbad"] < 2,
# #     res["elong"] < 1.8,
# #     res["magdiff"] < mag_diff,
# #     res["magdiff"] > -mag_diff,
# #     res["fwhm"] < 4.,
# #     res["sumrat"] > 0.8,
# #     res["chipsf"] < 10.0,
# # ]

# masks = [
#     res["nbad"] < 2,
#     res["chipsf"] < 3.0,
#     res["sumrat"] > 0.7,
#     res["fwhm"] < 10.,
#     # res["magdiff"] < 2.0,
#     res["magdiff"] < 1.6,
#     res["magdiff"] > -1.0,
#     res["mindtoedge"] > 50.0,
# ]
# mask = np.ones(len(res), dtype=bool)
# for new_mask in masks:
#     mask *= new_mask
    
# print(f"{np.sum(mask)} / {len(mask)} detections pass all cuts")
# cut = res[mask]

cut = apply_clean_filter(res)
# cut

In [15]:
ann_fields = [
    "ra", "dec", "fid", "field",  "candid",
    "neargaia", 'neargaiabright', 'maggaia', 'maggaiabright', 
    "distpsnr1", "tmjmag1",
    "nneg", "elong", "fwhm",  "sumrat",
    "magzpsci", "magap", "magapbig", "magdiff", "magfromlim",
    "classtar", "nbad", "nmtchps", "scorr", "chipsf", "mindtoedge"
]


populate_night(night, ann_fields=ann_fields)

100%|█████████████████████████████████████████| 486/486 [01:28<00:00,  5.48it/s]


In [8]:
# outpath = get_pdf_path(night)

# print(f"Saving PDF to {outpath}")

# with PdfPages(outpath) as pdf:
#     for i, row in tqdm(cut.iterrows(), total=len(cut)):
#         generate_single_page(row, ann_fields=ann_fields)
#         pdf.savefig()
#         plt.close()

In [12]:
# cut = cut.iloc[:30]


  0%|                                                   | 0/486 [00:00<?, ?it/s]


NameError: name 'get_single_pdf_path' is not defined

In [9]:
from mirar.database.transactions import select_from_table
from mirar.database.constraints import DBQueryConstraints

def get_night_db_entries(night, include_classified: bool = False):
    constraint = DBQueryConstraints("night", night, "=")

    if not include_classified:
        constraint.add_constraint("humanclass", 1, "<")

    return select_from_table(constraint, RealBogus.sql_model)

df = get_night_db_entries(night)
# def get_entries_for_night(night):


In [10]:
df

,candid,night,objectid,pdfpath,avropath,humanclass,comment


In [11]:
import pandas as pd
from mirar.database.transactions import _update_database_entry
from mirar.database.constraints import DBQueryConstraints

from winterdrb.models import RB_CLASS


def classify_source(candid, new_classification: int):

    assert new_classification in RB_CLASS

    con = DBQueryConstraints("candid", candid, "=")

    update_dict = {"humanclass": new_classification}

    _update_database_entry(update_dict, con, RealBogus.sql_model)

In [ ]:
classify_source(94355, 2)